In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import requests
import time
import pandas as pd
import csv

In [2]:
# ChromeDriver 옵션 설정
def initialize_webdriver():
    options = Options()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 제어 메시지 숨기기
    return webdriver.Chrome(options=options)

#### 가게 상세 페이지 리뷰 크롤링함수

In [6]:
def extract_visitor_reviews(driver, max_reviews=100):
    reviews = []  # 방문자 리뷰 데이터를 저장할 리스트
    review_count = 0  # 현재까지 수집된 리뷰의 개수
    visited_reviews = set()  # 이미 수집한 리뷰를 추적하기 위한 set (중복 방지)

    try:
        while review_count < max_reviews:
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
            
            # 첫 번째 페이지에서 리뷰 크롤링
            review_elements = soup.select("ul.list_evaluation > li")
            
            if not review_elements:
                print("리뷰가 더 이상 존재하지 않거나 페이지에 리뷰가 없습니다.")
                break  # 리뷰가 없는 경우 종료

            new_reviews = 0  # 새로 로드된 리뷰 수

            # 새로운 리뷰만 수집 (이미 수집된 리뷰는 제외)
            for review_element in review_elements:
                if review_count >= max_reviews:
                    break  # 최대 리뷰 개수에 도달하면 종료
                
                # 리뷰 텍스트 추출
                comment = review_element.select_one("p.txt_comment")
                comment_text = comment.get_text(strip=True) if comment else "리뷰 없음"
                
                if comment_text not in visited_reviews:
                    reviews.append(comment_text)
                    visited_reviews.add(comment_text)  # 새로 수집한 리뷰를 기록
                    review_count += 1
                    new_reviews += 1  # 새로 추가된 리뷰 수 증가

            if new_reviews == 0:
                print("새로운 리뷰가 더 이상 추가되지 않았습니다.")
                break  # 새 리뷰가 없으면 종료

            # 리뷰 개수가 충분치 않으면 "후기 더보기" 버튼을 클릭하여 더 많은 리뷰 로드
            if review_count < max_reviews:
                try:
                    # "후기 더보기" 버튼이 있을 경우 클릭
                    load_more_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.link_more > span.txt_more"))
                    )
                    # 버튼 클릭
                    driver.execute_script("arguments[0].click();", load_more_button)
                    time.sleep(3)  # 페이지 로딩 대기
                    print(f"{review_count}개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...")
                except Exception as e:
                    print(f"후기 더보기 버튼 클릭 중 오류 발생: {e}")
                    break  # "더보기" 버튼이 없거나 클릭이 안 되는 경우 종료

        # 100개 이하일 경우, 긁힌 리뷰 수만큼 반환
        if review_count < max_reviews:
            print(f"수집된 리뷰가 {review_count}개로 종료되었습니다.")
            return reviews

        return reviews

    except Exception as e:
        print(f"방문자 리뷰 수집 중 오류 발생: {e}")
        return ["후기가 제공되지 않는 가게입니다."]

#### csv 파일 저장 함수

In [7]:
def save_reviews_to_csv(reviews, store_name):
    """
    방문자 리뷰 데이터를 CSV 파일로 저장합니다.

    Args:
        reviews (list): 리뷰 데이터 리스트.
        store_name (str): 가게 이름.

    Returns:
        None
    """
    try:
        # 저장할 파일명
        filename = f"{store_name}_reviews.csv"

        # CSV 저장
        with open(filename, mode="w", encoding="utf-8-sig", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Review"])  # 헤더 작성
            for review in reviews:
                writer.writerow([review])  # 각 리뷰를 한 행에 저장

        print(f"리뷰 파일 '{filename}'이 저장되었습니다.")
    except Exception as e:
        print(f"리뷰 CSV 저장 중 오류 발생: {e}")

In [10]:
# 메인 실행
if __name__ == "__main__":
    driver = initialize_webdriver()
    try:
        # 1. 카카오맵 접속
        print("카카오맵 접속 중...")
        driver.get("https://map.kakao.com/")
        time.sleep(2)
        print("카카오맵 접속 완료!")

        # 2. 검색어 입력
        place_name = "크치치킨"  # 검색할 가게명
        print("검색창에 가게명 입력 중...")
        search_box = driver.find_element(By.ID, "search.keyword.query")
        search_box.send_keys(place_name)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)
        print(f"'{place_name}' 검색 완료!")

        # 3. 투명 레이어 제거
        try:
            dimmed_layer = driver.find_element(By.ID, "dimmedLayer")
            driver.execute_script("arguments[0].style.display = 'none';", dimmed_layer)
            print("투명 레이어 제거 완료!")
        except Exception as e:
            print("투명 레이어가 존재하지 않습니다. 계속 진행합니다.")

        try:
        # 현재 페이지 소스를 BeautifulSoup로 파싱
            print("검색 결과 파싱 중...")
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
    
            # 검색 결과에서 첫 번째 항목의 "상세보기" 버튼 링크 추출
            first_result = soup.select_one("#info\\.search\\.place\\.list > li.PlaceItem.clickArea a.moreview")
            if first_result:
                detail_url = first_result['href']
                print(f"첫 번째 상세보기 링크: {detail_url}")
    
                # 상세 페이지로 이동
                driver.get(detail_url)
                time.sleep(2)
                print("상세 페이지로 이동 완료!")
            else:
                print("검색 결과에서 첫 번째 항목을 찾을 수 없습니다.")
    
        except Exception as e:
            print(f"검색 결과 또는 상세 페이지 이동 중 오류 발생: {e}")

        # 리뷰 크롤링
        reviews = extract_visitor_reviews(driver, max_reviews=100)
        if reviews:
            print("크롤링 결과 저장 중...")
            save_reviews_to_csv(reviews, store_name=place_name)  # 리뷰 데이터를 CSV 파일로 저장
        else:
            print("리뷰 크롤링 결과 없음.")

    except Exception as e:
        print(f"오류 발생: {e}")
    finally:
        driver.quit()
        print("브라우저 종료 완료.")

카카오맵 접속 중...
카카오맵 접속 완료!
검색창에 가게명 입력 중...
'크치치킨' 검색 완료!
투명 레이어 제거 완료!
검색 결과 파싱 중...
첫 번째 상세보기 링크: https://place.map.kakao.com/641407772
상세 페이지로 이동 완료!
3개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...
7개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...
11개의 리뷰를 수집했습니다. 더 많은 리뷰를 로딩 중...
새로운 리뷰가 더 이상 추가되지 않았습니다.
수집된 리뷰가 11개로 종료되었습니다.
크롤링 결과 저장 중...
리뷰 파일 '크치치킨_reviews.csv'이 저장되었습니다.
브라우저 종료 완료.
